In [ ]:
## earlier attempt
## doesn't work because multiple hooks get added to the same module instance :(

def forward_tree(
    module: nn.Module, inputs: Any
) -> tuple[dict[str, Any], dict[str, Any], dict[str, Any]]:

    input_tree: dict[str, Any] = {}
    output_tree: dict[str, Any] = {}
    module_tree: dict[str, nn.Module] = {}

    def make_hook(key: str) -> Callable[[nn.Module, Any, Any], None]:
        def hook(mod: nn.Module, inp: Any, out: Any) -> None:
            # inp[0] here restricts us to forward() first argument
            # so this only works with single argument forward() functions
            if key not in input_tree:
                print("hook called on ", key, type(key))
                input_tree[key] = inp[0]
                output_tree[key] = out
                module_tree[key] = mod
        return hook


    # Register forward hooks to every module recursively
    hooks = []
    for key, mod in module.named_modules(remove_duplicate=False, prefix="root"):
        print(".")
        hook = make_hook(key)

        hooks.append(mod.register_forward_hook(hook))

    # Evaluate the full model, then remove all hooks
    try:
        _ = module(inputs)
    finally:
        for h in hooks:
            h.remove()

    return input_tree, output_tree, module_tree

In [ ]:
import torch
import torch.nn as nn

from torchlensmaker.full_forward import forward_tree

class Group(nn.Module):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b

    def forward(self, x):
        return x + 10

class Mod(nn.Module):
    def forward(self, x):
        return x + 1

lens = Mod()

model = nn.Sequential(
    lens,
    lens,
    nn.Sequential(
        lens,
        lens
    ),
    Group(lens, lens),
)

print(model)

input_tree, output_tree, module_tree = forward_tree(model, 0)

print(input_tree)
print(output_tree)
print(module_tree)